# Discriminative Classifiers: K-Nearest Neighbours

In this practical, we will first manually implement k-nearest neighbours. We will explore how the algorithm constructs decision boundaries and how it classifies data. We will visualise these boundaries for our own implementation, before using `sklearn` implentations to compare and contrast with our own.

Then, we will move to using `sklearn` for a more traditional data science task, using a real dataset.

# Implementing KNN

Recall that the KNN algorithm relies on the concept of distance between two items in order to quantify how similar they are. Input data is represented as an array of features, which locates each item within a vector space. We will use data with two dimensions, longitude and latitude, which will make it easy to visualise.

We will need four functions:

1. `euclidean_distance(p1, p2)` - given a pair of (x,y) coordinates, calculate how close they are using Euclidean distance: $$ \sqrt{(x_2-x_1)^2+(y_2-y_1)^2} $$
2. `manhattan_distance(p1, p2)` - calculate $$ | x_1 - x_2 | + | y_1 - y_2 | $$
3. `get_neighbours(new_point, known_points, distance_function)` - given a single coordinate, calculate how close it is to all other points using the provided distance function
4. `classify(neighbours_list, k)` - given a list of points with their distances and their class label, determine which class is most common in the `k` nearest neighbours

The classification process will work as follows:

1. For a new point with no class label, get the distance from it to every known point
2. From that list of distances, take the `k` nearest ones
3. From that subset, find out which class is the most common and use it to label the new point

Below are templates for the functions, which you will complete.

## The distance functions

Complete the two functions below using the docstring as a guide.

(Note: to calculate the absolute value for Manhattan distance, use the `abs()` function.)

In [1]:
from math import sqrt  # For calculating square root


def euclidean_distance(point1, point2):
    """
    Inputs:
        point1 : a tuple of (lon, lat) coordinates
        point2 : a tuple of (lon, lat) coordinates

    Returns:
        distance : a float, the distance between the points
    """

    # Your code here...

    distance = sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

    return distance


In [2]:
def manhattan_distance(point1, point2):
    """
    Inputs:
        point1 : a tuple of (lon, lat) coordinates
        point2 : a tuple of (lon, lat) coordinates

    Returns:
        distance : a float, the distance between the points
    """

    # Your code here...

    distance = abs(point1[0] - point2[0]) + abs(point1[1] - point2[1])

    return distance



## Testing the `distance` functions

Run the cell below to test your functions are working as expected.

In [3]:
assert euclidean_distance((0, 0), (0, 0)) == 0
assert euclidean_distance((0, 0), (-1, 0)) == 1
assert euclidean_distance((0, 0), (2, 0)) == 2
assert euclidean_distance((0, 0), (1, 1)) == 1.4142135623730951
assert euclidean_distance((1233, 376.4), (-213.2, 0.03)) == 1494.372382272906

assert manhattan_distance((0, 0), (0, 0)) == 0
assert manhattan_distance((0, 0), (-1, 0)) == 1
assert manhattan_distance((0, 0), (2, 0)) == 2
assert manhattan_distance((0, 0), (1, 1)) == 2
assert manhattan_distance((1233, 376.4), (-213.2, 0.03)) == 1822.5700000000002

print("All good!")

## The `get_neighbours` function

This should loop through a list of labeled points and calculate the distance between each one and a new unlabeled point, using your `distance` function.

(Note: the returned list should be sorted, such that the first item is the nearest and so on. Use `sorted()` on a list. To sort them by the third item in each sub-item, use `sorted(your_list, key=itemgetter(2))` - remember that Python starts counting at 0, not 1.)

In [4]:
from operator import (
    itemgetter,
)  # For sorting lists by particular items inside sub-items



def get_neighbours(new_point, known_points, distance_function):
    """
    Inputs:
        new_point : a tuple of (lon, lat) coordinates
        known_points : a list of tuples of (lon, lat, label)
        distance_function : a function to calculate distance
    Returns:
        neighbours : a sorted list of tuples with ((lon, lat), label, distance)
    """

    # Your code here...

    neighbours = []

    for lon, lat, label in known_points:
        d = distance_function(new_point, (lon, lat))

        neighbours.append(((lon, lat), label, d))


    neighbours = sorted(neighbours, key=itemgetter(2))

    return neighbours


## Testing the `distance` function

Run the cell below to test your function is working as expected.

In [5]:
known = [(0, 0, "A"), (1, 1, "A"), (3, 4, "B"), (5, 4, "B"), (1.5, 1.5, "C")]
new = (2, 2)

assert get_neighbours(new, known, euclidean_distance) == [
    ((1.5, 1.5), "C", 0.7071067811865476),
    ((1, 1), "A", 1.4142135623730951),
    ((3, 4), "B", 2.23606797749979),
    ((0, 0), "A", 2.8284271247461903),
    ((5, 4), "B", 3.605551275463989),
]

assert get_neighbours(new, known, manhattan_distance) == [
    ((1.5, 1.5), "C", 1.0),
    ((1, 1), "A", 2),
    ((3, 4), "B", 3),
    ((0, 0), "A", 4),
    ((5, 4), "B", 5),
]

print("All good!")

## The `classify` function

This function should take the output of `get_neighbours` and calculate which class is the most common in the nearest `k` items.

`collections.Counter` can turn a list of items into a dictionary of item:count.

`Counter(['A', 'B', 'B'])` gives you `{'A':1, 'B':2}`

To get the top item, use the `.most_common()` method, which returns a sorted list of item/count pairs. The first item is the most common item.

In [6]:
from collections import Counter

some_list = list("AUHSUGQWURBUQWVYFVQFQFQIHIUQBF")

counts = Counter(some_list)
print(f"The counter: {counts}\n")

top_item = counts.most_common()
print(f"Items sorted by their counts: {top_item}\n")
print(f"Most common item and its count: {top_item[0]}\n")
print(f"Most common item's name: {top_item[0][0]}\n")
print(f"Most common item's count: {top_item[0][1]}")

In [7]:
# Complete the function below.
# The first part checks to make sure k is a valid value,
# based on the data the function is passed.


def classify(neighbours_list, k):
    """
    Inputs:
        neighbours_list : a list of ((lon, lat), label, distance)
        k : the number of neighbours to use for classification
    Returns:
        label : the most commonly observed label of the top k items in neighbours
    """
    if k <= 0:
        raise ValueError("k too low: must be > 0")
    if k > len(neighbours_list):
        raise ValueError("k too high: must be <= the number of neighbours")

    # Your code here...
    top_neighbours = neighbours_list[0:k]
    top_classes = [neighbour[1] for neighbour in top_neighbours]
    counts = Counter(top_classes)
    top_item = counts.most_common()[0]
    label = top_item[0]
    return label


## Testing

In [8]:
known = [(0, 0, "A"), (1, 1, "A"), (3, 4, "B"), (5, 4, "B"), (1.5, 1.5, "C")]
new = (0.5, 0.5)

neighbours_euc = get_neighbours(new, known, euclidean_distance)
assert classify(neighbours_euc, 3) == "A"

neighbours_man = get_neighbours(new, known, manhattan_distance)
assert classify(neighbours_man, 3) == "A"

print("All good!")

## Putting it all together

Putting this all together, we can now do KNN classification with the following workflow.

(It's not as elegant as `sklearn` but it works!)

In [9]:
train_data = [(1, 1, "A"), (1, 1.5, "A"), (2, 2, "A"), (4, 5, "B"), (8, 4, "B")]

new_data = [(0.5, 0.5), (9.3, 9.0), (5.1, 5.6)]

for point in new_data:
    neighbours = get_neighbours(point, train_data, euclidean_distance)
    label = classify(neighbours, k=3)

    print(f"Item: {point} Prediction: {label}")

## KNN: assigning towns to regions

Imagine you are a government official, in charge of assigning new towns to a region.

This job was easy until someone lost all the boundary maps. Now, nobody knows what the geographical properties of each region actually are.

Fortunately, you **do** know which region all the existing towns are in, as well as the coordinates of all those towns.

You decide that the best approach is assign new towns to a region based on which other towns it is nearest to.

In [10]:
import pandas as pd

data = pd.read_csv("data/uk-towns-sample.csv")

train_data = data[data["kind"] == "Train"]
test_data = data[data["kind"] == "Test"]

train_data.head()

In [11]:
# Get the data into the expected format for our functions

train_data = list(
    zip(train_data.lon.tolist(), train_data.lat.tolist(), train_data.region.tolist())
)
test_data = list(
    zip(test_data.lon.tolist(), test_data.lat.tolist(), test_data.region.tolist())
)

With your KNN functions, write a loop that goes through the first 3 items of `test_data` and predicts a region for them.

Get predictions for Euclidean distance, at `k=3` and `k=300`.

What do you observe? What happens when `k` is large? Why do you think that is?

In [12]:
# Your code here...

for lon, lat, region in test_data[0:3]:
    point = (lon, lat)
    neighbours = get_neighbours(point, train_data, euclidean_distance)
    label_3 = classify(neighbours, k=3)
    label_300 = classify(neighbours, k=300)

    print(f"Expected region: {region}")
    print(f"\t Prediction \n\t k=3: {label_3} \n\t k=300 {label_300}\n")

# Accuracy is perfect at k=3 but at k=300 it is zero. This is likely because using too many neighbours simply selects the region containing the most towns.



## Evaluating the predictions

The cell below will use the test data to evaluate performance at different `k` values. As you can see, setting `k` too high or too low hurts performance on unseen data.

Note that our implementation takes around 1 second to run all these predictions.

In [ ]:
%%time

from sklearn.metrics import f1_score

for k in [1, 3, 5, 10, 20, 100, 300]:
    predictions = []
    expected = []

    for lon, lat, region in test_data:
        point = (lon, lat)

        neighbours = get_neighbours(point, train_data, euclidean_distance)
        label = classify(neighbours, k=k)

        predictions.append(label)
        expected.append(region)

    print(
        f'F1 score for k={k}\t: {f1_score(expected, predictions, zero_division=0, average="weighted"):.3f}'
    )

## Recreating the map

How could you use the data and the model to create a new map, showing the regional boundaries? How would it compare to the "true" map?

In [ ]:
# Your thoughts below...

# You could classify every single lon/lat point on a map grid. This should show the structure of the boundaries.
# It should look fairly similar to the original map, but the outlines of the regions are not likely to be very high resolution.



Running the cell below will do the following

1. Determine the min/max lon/lat points
2. Use these to create a grid of (x,y) points
3. Classify all these points using the KNN model
4. Colour each point according to prediction
5. Plot predictions and towns

(It will take maybe a minute or two, because the KNN prediction process is not optimised! But there is a little progress bar, to show you that things are happening.)

In [ ]:
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
from tqdm.notebook import tqdm

colour_vals = np.linspace(0, 1, 96)
np.random.seed(5)
np.random.shuffle(colour_vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.hsv(colour_vals))


def plot(k):
    # Make coords for predicting
    min_lon, max_lon = (
        min(train_data, key=itemgetter(0))[0],
        max(train_data, key=itemgetter(0))[0],
    )
    min_lat, max_lat = (
        min(train_data, key=itemgetter(1))[1],
        max(train_data, key=itemgetter(1))[1],
    )

    xx, yy = np.meshgrid(
        np.arange(min_lon, max_lon, 0.1), np.arange(min_lat, max_lat, 0.1)
    )

    # Do the predictions
    grid_predictions = []
    for point in tqdm(zip(xx.flatten(), yy.flatten()), total=len(xx.flatten())):
        neighbours = get_neighbours(point, train_data, euclidean_distance)
        label = classify(neighbours, k=k)
        grid_predictions.append(label)
    grid_predictions = np.array(grid_predictions).reshape(xx.shape)

    # Need to be able to convert between text and numerical labels, to plot properly
    p_to_i = {p: i for i, p in enumerate(set([i[2] for i in train_data]))}
    i_to_p = {i: p for i, p in enumerate(set([i[2] for i in train_data]))}

    # Plot each grid prediction
    plt.pcolormesh(
        xx,
        yy,
        np.array([p_to_i[p] for p in grid_predictions.flatten()]).reshape(xx.shape),
        cmap=cmap,
        shading="auto",
    )

    # Plot the towns, coloured by region
    x_vals = [i[0] for i in train_data]
    y_vals = [i[1] for i in train_data]
    colors = [p_to_i[i[2]] for i in train_data]
    plt.scatter(x_vals, y_vals, c=colors, cmap=cmap, edgecolor="k", s=30)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title(f"k={k}")

    # Output a classification report


f, a = plt.subplots(1, 3, figsize=(36, 18))

for i, k in enumerate([5, 10, 20]):
    plt.sca(a[i])
    plot(k)

## Interpreting the boundaries

In the three plots, we used a `k` of 3, 10 and 20. What do you observe in terms of the decision boundaries?

What problems do you anticipate in terms of using this as a map of the country's regional boundaries?

In [ ]:
# Your thoughts below...

# When k is large, a lot of the points are misclassified. You can tell this from the dot colour not matching the area colour.
# You can see a lot of misclassified items where boundaries meet, as k gets bigger. Regions in Northern Ireland are especially poorly delineated.
# This would make a decent map if it were able to tell the difference between land and sea! You can see the layout of the UK in the dots, but the model has no conception of where it is possible for towns to go.
# One way around this would be to only generate predictions for coordinates we know are on land.



# KNN: sklearn

Now let's compare our implementation to that in `sklearn`. Run the cell below. You'll see it only takes 100ms, compared to 1s for ours!

In [ ]:
%%time

from sklearn.neighbors import KNeighborsClassifier

data = pd.read_csv("data/uk-towns-sample.csv")

train_data = data[data.kind == "Train"]
test_data = data[data.kind == "Test"]

for our_score, k in zip(
    [0.898, 0.912, 0.891, 0.846, 0.772, 0.382, 0.095], [1, 3, 5, 10, 20, 100, 300]
):
    knn = KNeighborsClassifier(n_neighbors=k)

    knn.fit(train_data[["lon", "lat"]], train_data["region"])
    sklearn_score = knn.score(test_data[["lon", "lat"]], test_data["region"])

    print(f"k={k}")
    print(f"sklearn F1 score: {sklearn_score:.3f}")
    print(f"Our F1 score: {our_score}")
    if our_score - sklearn_score > 0:
        print(f"Ours was better by {our_score - sklearn_score:.3f}\n\n")
    else:
        print(f"Sklearn was better by {our_score - sklearn_score:.3f}\n\n")

# Conclusion

In this practical, you implemented and explored supervised algorithms for discriminative classification. You implemented k-nearest neighbours yourself, but also used `sklearn`'s (much faster) version of it.

If you want to extend the work here, you could explore how the different distance measures impacts KNN performance - can you generate a better boundary map?